# Capítulo 07 - Tipos de Dados no DuckDB

Este notebook explora o sistema de tipos do DuckDB, desde tipos básicos até estruturas complexas.

## 📚 Tópicos Abordados:
1. Tipos Numéricos (inteiros e ponto flutuante)
2. Tipos de Texto (VARCHAR, TEXT, STRING)
3. Tipos Booleanos
4. Tipos Temporais (DATE, TIME, TIMESTAMP)
5. Tipos Binários (BLOB)
6. UUID
7. JSON
8. Arrays e Listas
9. Structs (registros)
10. Maps
11. Union Types
12. Conversões de Tipos
13. Exemplos Práticos

## 1. Setup e Preparação

In [ ]:
import duckdb
import json

# Criar conexão
con = duckdb.connect(':memory:')

print(f"DuckDB versão: {duckdb.__version__}")
print("✓ Conexão criada!")

## 2. Tipos Numéricos Inteiros

### 2.1 Inteiros Com Sinal

In [ ]:
con.execute("""
    CREATE TABLE numeros (
        pequeno TINYINT,      -- -128 a 127 (1 byte)
        medio INTEGER,        -- -2.1B a 2.1B (4 bytes)
        grande BIGINT,        -- -9.2E18 a 9.2E18 (8 bytes)
        enorme HUGEINT        -- 128-bit integer
    )
""")

con.execute("""
    INSERT INTO numeros VALUES 
    (127, 1000000, 9223372036854775807, 123456789012345678901234567890)
""")

print("Tabela 'numeros' criada:")
con.execute("SELECT * FROM numeros").fetchdf()

### 2.2 Inteiros Sem Sinal (Unsigned)

In [ ]:
con.execute("""
    CREATE TABLE contadores (
        id UINTEGER,          -- 0 a 4.2B
        visitas UBIGINT       -- 0 a 18.4E18
    )
""")

con.execute("""
    INSERT INTO contadores VALUES 
    (1, 18446744073709551615)
""")

print("Contadores (unsigned):")
con.execute("SELECT * FROM contadores").fetchdf()

### 2.3 Ponto Flutuante

In [ ]:
con.execute("""
    CREATE TABLE medicoes (
        temperatura FLOAT,    -- 32-bit (precisão simples)
        precisao DOUBLE       -- 64-bit (precisão dupla)
    )
""")

con.execute("""
    INSERT INTO medicoes VALUES 
    (36.5, 3.14159265358979323846)
""")

print("Ponto flutuante:")
result = con.execute("SELECT * FROM medicoes").fetchdf()
print(result)

print("\n💡 FLOAT tem menos precisão que DOUBLE")

### 2.4 DECIMAL (Precisão Fixa)

In [ ]:
# DECIMAL(precisão, escala)
con.execute("""
    CREATE TABLE financeiro (
        preco DECIMAL(10, 2),  -- 10 dígitos, 2 decimais
        taxa DECIMAL(5, 4)     -- 5 dígitos, 4 decimais
    )
""")

con.execute("""
    INSERT INTO financeiro VALUES 
    (12345.67, 0.0525)
""")

print("Valores decimais (precisão fixa):")
con.execute("SELECT * FROM financeiro").fetchdf()

print("\n✓ DECIMAL mantém precisão exata (ideal para dinheiro!)")

## 3. Tipos de Texto

### 3.1 VARCHAR, TEXT e STRING

In [ ]:
con.execute("""
    CREATE TABLE textos (
        nome VARCHAR,       -- Texto variável
        descricao TEXT,     -- Alias de VARCHAR
        comentario STRING   -- Alias de VARCHAR
    )
""")

con.execute("""
    INSERT INTO textos VALUES
    ('Alice', 'Desenvolvedora', 'Especialista em Python')
""")

print("Tipos de texto (todos equivalentes):")
con.execute("SELECT * FROM textos").fetchdf()

print("\n💡 VARCHAR, TEXT e STRING são sinônimos!")

### 3.2 VARCHAR com Limite

In [ ]:
# VARCHAR com limite de tamanho
con.execute("""
    CREATE TABLE usuarios (
        username VARCHAR(50),
        email VARCHAR(255)
    )
""")

con.execute("""
    INSERT INTO usuarios VALUES
    ('alice_silva', 'alice@example.com')
""")

print("VARCHAR com limite:")
con.execute("SELECT * FROM usuarios").fetchdf()

## 4. Tipos Booleanos

In [ ]:
con.execute("""
    CREATE TABLE configuracoes (
        ativo BOOLEAN,
        verificado BOOL,      -- Alias
        publico LOGICAL       -- Alias
    )
""")

# Inserir com true/false
con.execute("""
    INSERT INTO configuracoes VALUES 
    (true, false, true)
""")

# Inserir com 1/0
con.execute("""
    INSERT INTO configuracoes VALUES 
    (1, 0, 1)
""")

print("Valores booleanos:")
con.execute("SELECT * FROM configuracoes").fetchdf()

print("\n💡 1 = true, 0 = false")

## 5. Tipos Temporais

### 5.1 DATE

In [ ]:
con.execute("""
    CREATE TABLE eventos (
        nome VARCHAR,
        data DATE
    )
""")

con.execute("""
    INSERT INTO eventos VALUES
    ('Lançamento', '2024-01-15'),
    ('Conferência', DATE '2024-06-20')
""")

print("Datas:")
con.execute("SELECT * FROM eventos").fetchdf()

# Operações com datas
print("\nOperações:")
result = con.execute("""
    SELECT 
        nome,
        data,
        data + INTERVAL '7 days' AS uma_semana_depois,
        date_diff('day', data, CURRENT_DATE) AS dias_desde_evento
    FROM eventos
""").fetchdf()
print(result)

### 5.2 TIME

In [ ]:
con.execute("""
    CREATE TABLE horarios (
        abertura TIME,
        fechamento TIME
    )
""")

con.execute("""
    INSERT INTO horarios VALUES
    ('09:00:00', '18:00:00'),
    (TIME '14:30:00', TIME '23:59:59')
""")

print("Horários:")
result = con.execute("SELECT * FROM horarios").fetchdf()
print(result)

### 5.3 TIMESTAMP

In [ ]:
con.execute("""
    CREATE TABLE logs (
        mensagem VARCHAR,
        timestamp TIMESTAMP
    )
""")

con.execute("""
    INSERT INTO logs VALUES
    ('Início', '2024-01-15 10:30:00'),
    ('Fim', TIMESTAMP '2024-01-15 18:45:30.123456')
""")

print("Timestamps:")
result = con.execute("SELECT * FROM logs").fetchdf()
print(result)

# Extrair componentes
print("\nExtrair partes:")
result = con.execute("""
    SELECT 
        mensagem,
        EXTRACT(year FROM timestamp) AS ano,
        EXTRACT(month FROM timestamp) AS mes,
        EXTRACT(hour FROM timestamp) AS hora
    FROM logs
""").fetchdf()
print(result)

### 5.4 TIMESTAMP WITH TIME ZONE

In [ ]:
con.execute("""
    CREATE TABLE eventos_globais (
        evento VARCHAR,
        quando TIMESTAMPTZ  -- Alias de TIMESTAMP WITH TIME ZONE
    )
""")

con.execute("""
    INSERT INTO eventos_globais VALUES
    ('Webinar', '2024-01-15 10:00:00-03:00'),  -- Horário de Brasília
    ('Meeting', '2024-01-15 14:00:00+00:00')   -- UTC
""")

print("Eventos com timezone:")
result = con.execute("SELECT * FROM eventos_globais").fetchdf()
print(result)

print("\n💡 Armazenado como UTC, exibido no timezone local")

### 5.5 INTERVAL

In [ ]:
# Intervalos de tempo
print("Operações com INTERVAL:")
result = con.execute("""
    SELECT
        '2024-01-15'::DATE + INTERVAL '7 days' AS uma_semana_depois,
        '2024-01-15 10:00:00'::TIMESTAMP + INTERVAL '2 hours 30 minutes' AS mais_tarde,
        INTERVAL '1 year 2 months 3 days' AS periodo
""").fetchdf()
print(result)

## 6. Tipos Binários (BLOB)

In [ ]:
con.execute("""
    CREATE TABLE arquivos (
        nome VARCHAR,
        conteudo BLOB
    )
""")

# Inserir dados binários
con.execute("""
    INSERT INTO arquivos VALUES
    ('imagem.png', '\\xAB\\xCD\\xEF'::BLOB),
    ('doc.pdf', '\\x25\\x50\\x44\\x46'::BLOB)
""")

print("Arquivos binários:")
result = con.execute("""
    SELECT 
        nome,
        octet_length(conteudo) AS tamanho_bytes
    FROM arquivos
""").fetchdf()
print(result)

print("\n💡 BLOB armazena dados binários")

## 7. UUID

In [ ]:
con.execute("""
    CREATE TABLE ids_usuarios (
        id UUID,
        nome VARCHAR
    )
""")

# Gerar UUID
con.execute("""
    INSERT INTO ids_usuarios VALUES
    (uuid(), 'Alice'),
    (uuid(), 'Bob'),
    (uuid(), 'Charlie')
""")

print("UUIDs gerados:")
result = con.execute("SELECT * FROM ids_usuarios").fetchdf()
print(result)

print("\n✓ uuid() gera identificadores únicos")

## 8. JSON

In [ ]:
con.execute("""
    CREATE TABLE documentos (
        id INTEGER,
        dados JSON
    )
""")

con.execute("""
    INSERT INTO documentos VALUES
    (1, '{"nome": "Alice", "idade": 30}'),
    (2, '{"produto": "Laptop", "preco": 1200.50, "specs": {"ram": "16GB"}}')
""")

print("Documentos JSON:")
result = con.execute("SELECT * FROM documentos").fetchdf()
print(result)

# Consultar JSON
print("\nExtrair campos JSON:")
result = con.execute("""
    SELECT
        id,
        dados->>'nome' AS nome,
        dados->>'idade' AS idade,
        dados->>'produto' AS produto
    FROM documentos
""").fetchdf()
print(result)

## 9. Arrays e Listas

### 9.1 Array (Tamanho Fixo)

In [ ]:
# Array com tamanho fixo (3 elementos)
con.execute("""
    CREATE TABLE vetores (
        coordenadas INTEGER[3]
    )
""")

con.execute("""
    INSERT INTO vetores VALUES 
    ([1, 2, 3]),
    ([10, 20, 30])
""")

print("Arrays (tamanho fixo):")
result = con.execute("SELECT * FROM vetores").fetchdf()
print(result)

### 9.2 Lista (Tamanho Variável)

In [ ]:
# Lista com tamanho variável
con.execute("""
    CREATE TABLE listas (
        numeros INTEGER[]
    )
""")

con.execute("""
    INSERT INTO listas VALUES
    ([1, 2, 3]),
    ([1, 2]),           -- Tamanhos diferentes OK
    ([10, 20, 30, 40, 50])
""")

print("Listas (tamanho variável):")
result = con.execute("SELECT * FROM listas").fetchdf()
print(result)

# Acessar elementos (indexação 1-based!)
print("\nAcessar primeiro elemento:")
result = con.execute("""
    SELECT 
        numeros,
        numeros[1] AS primeiro,
        len(numeros) AS tamanho
    FROM listas
""").fetchdf()
print(result)

print("\n⚠️ Indexação começa em 1, não 0!")

### 9.3 Funções de Lista

In [ ]:
print("Funções de lista:")
result = con.execute("""
    SELECT
        [1, 2, 3, 4, 5] AS lista,
        len([1, 2, 3, 4, 5]) AS tamanho,
        list_contains([1, 2, 3, 4, 5], 3) AS contem_3,
        list_sum([1, 2, 3, 4, 5]) AS soma,
        list_avg([1, 2, 3, 4, 5]) AS media,
        list_max([1, 2, 3, 4, 5]) AS maximo
""").fetchdf()
print(result)

## 10. STRUCT (Registros)

### 10.1 Criar e Usar Structs

In [ ]:
con.execute("""
    CREATE TABLE pessoas (
        id INTEGER,
        endereco STRUCT(rua VARCHAR, cidade VARCHAR, cep VARCHAR)
    )
""")

con.execute("""
    INSERT INTO pessoas VALUES
    (1, {'rua': 'Rua A', 'cidade': 'São Paulo', 'cep': '01000-000'}),
    (2, {'rua': 'Rua B', 'cidade': 'Rio de Janeiro', 'cep': '20000-000'})
""")

print("Tabela com STRUCT:")
result = con.execute("SELECT * FROM pessoas").fetchdf()
print(result)

### 10.2 Acessar Campos de Struct

In [ ]:
# Notação de ponto
print("Acessar campos do STRUCT:")
result = con.execute("""
    SELECT
        id,
        endereco.rua,
        endereco.cidade,
        endereco.cep
    FROM pessoas
""").fetchdf()
print(result)

# Ou usando colchetes
print("\nUsando colchetes:")
result = con.execute("""
    SELECT endereco['rua'] AS rua FROM pessoas
""").fetchdf()
print(result)

### 10.3 Structs Aninhados

In [ ]:
print("STRUCT aninhado:")
result = con.execute("""
    SELECT {
        'pessoa': {
            'nome': 'Alice',
            'idade': 30
        },
        'contato': {
            'email': 'alice@example.com',
            'telefone': '11-99999-9999'
        }
    } AS dados_complexos
""").fetchdf()
print(result)

## 11. MAP (Dicionário)

### 11.1 Criar e Usar Maps

In [ ]:
# MAP de chave → valor
con.execute("""
    CREATE TABLE configs (
        id INTEGER,
        opcoes MAP(VARCHAR, INTEGER)
    )
""")

# Criar MAP
con.execute("""
    INSERT INTO configs VALUES
    (1, map(['theme', 'font_size', 'line_height'], [1, 14, 20])),
    (2, map(['volume', 'brightness'], [75, 80]))
""")

print("Tabela com MAP:")
result = con.execute("SELECT * FROM configs").fetchdf()
print(result)

# Acessar valores
print("\nAcessar valores do MAP:")
result = con.execute("""
    SELECT
        id,
        opcoes['theme'] AS tema,
        opcoes['font_size'] AS tamanho_fonte
    FROM configs
    WHERE id = 1
""").fetchdf()
print(result)

## 12. UNION Types

In [ ]:
con.execute("""
    CREATE TABLE valores_mistos (
        id INTEGER,
        valor UNION(num INTEGER, texto VARCHAR)
    )
""")

con.execute("""
    INSERT INTO valores_mistos VALUES
    (1, union_value(num := 42)),
    (2, union_value(texto := 'Olá')),
    (3, union_value(num := 100))
""")

print("UNION types:")
result = con.execute("SELECT * FROM valores_mistos").fetchdf()
print(result)

# Extrair valor com base no tipo
print("\nExtrair valores:")
result = con.execute("""
    SELECT
        id,
        union_extract(valor, 'num') AS numero,
        union_extract(valor, 'texto') AS texto,
        union_tag(valor) AS tipo_atual
    FROM valores_mistos
""").fetchdf()
print(result)

## 13. Conversões de Tipos

### 13.1 CAST

In [ ]:
print("Conversões com CAST:")
result = con.execute("""
    SELECT
        CAST('42' AS INTEGER) AS str_para_int,
        CAST(3.14 AS INTEGER) AS float_para_int,
        '42'::INTEGER AS usando_dois_pontos,
        '2024-01-15'::DATE AS str_para_date,
        'true'::BOOLEAN AS str_para_bool
""").fetchdf()
print(result)

### 13.2 TRY_CAST (Seguro)

In [ ]:
# TRY_CAST retorna NULL em vez de erro
print("TRY_CAST (retorna NULL se falhar):")
result = con.execute("""
    SELECT
        TRY_CAST('abc' AS INTEGER) AS falha,
        TRY_CAST('42' AS INTEGER) AS sucesso,
        TRY_CAST('2024-99-99' AS DATE) AS data_invalida
""").fetchdf()
print(result)

print("\n💡 TRY_CAST evita erros!")

## 14. Exemplo Prático Completo

### 14.1 Tabela de Produtos (Tipos Complexos)

In [ ]:
con.execute("""
    CREATE TABLE produtos (
        id UINTEGER PRIMARY KEY,
        nome VARCHAR,
        preco DECIMAL(10, 2),
        estoque INTEGER,
        categorias VARCHAR[],
        detalhes STRUCT(
            peso DECIMAL(5, 2),
            dimensoes STRUCT(
                altura INTEGER,
                largura INTEGER,
                profundidade INTEGER
            )
        ),
        metadados JSON,
        criado_em TIMESTAMP,
        ativo BOOLEAN
    )
""")

con.execute("""
    INSERT INTO produtos VALUES (
        1,
        'Notebook',
        2499.99,
        50,
        ['Eletrônicos', 'Computadores', 'Portáteis'],
        {
            'peso': 1.5,
            'dimensoes': {'altura': 2, 'largura': 35, 'profundidade': 25}
        },
        '{"marca": "TechCo", "garantia": "2 anos"}',
        '2024-01-15 10:00:00',
        true
    )
""")

print("✓ Produto inserido!")

### 14.2 Consulta Complexa

In [ ]:
print("Consulta com tipos complexos:")
result = con.execute("""
    SELECT
        nome,
        preco,
        len(categorias) AS num_categorias,
        categorias[1] AS categoria_principal,
        detalhes.peso AS peso_kg,
        detalhes.dimensoes.altura AS altura_cm,
        detalhes.dimensoes.largura AS largura_cm,
        metadados->>'marca' AS marca,
        metadados->>'garantia' AS garantia,
        date_diff('day', criado_em, current_timestamp) AS dias_desde_criacao
    FROM produtos
    WHERE ativo = true
""").fetchdf()
print(result)

print("\n✓ Todos os tipos usados em uma query!")

## 15. Funções de Inspeção de Tipos

### 15.1 typeof()

In [ ]:
print("Inspecionar tipos:")
result = con.execute("""
    SELECT
        typeof(42) AS tipo_int,
        typeof('texto') AS tipo_varchar,
        typeof([1, 2, 3]) AS tipo_lista,
        typeof({'a': 1}) AS tipo_struct,
        typeof(3.14) AS tipo_double,
        typeof(true) AS tipo_bool
""").fetchdf()
print(result)

### 15.2 DESCRIBE

In [ ]:
# Ver tipos das colunas
print("Estrutura da tabela produtos:")
result = con.execute("DESCRIBE produtos").fetchdf()
print(result)

## 16. Trabalhando com NULL

### 16.1 NULL é Compatível com Todos os Tipos

In [ ]:
con.execute("""
    CREATE TABLE exemplo_null (
        num INTEGER,
        texto VARCHAR,
        lista INTEGER[]
    )
""")

con.execute("""
    INSERT INTO exemplo_null VALUES 
    (NULL, NULL, NULL),
    (42, 'texto', [1, 2, 3])
""")

print("Tabela com NULL:")
result = con.execute("SELECT * FROM exemplo_null").fetchdf()
print(result)

# Verificar NULL
print("\nTratar NULL:")
result = con.execute("""
    SELECT
        num IS NULL AS num_nulo,
        COALESCE(num, 0) AS num_com_default,
        IFNULL(texto, 'sem texto') AS texto_com_default
    FROM exemplo_null
""").fetchdf()
print(result)

## 🎯 Resumo do Capítulo

### ✅ Tipos Abordados:

**Numéricos:**
- TINYINT, INTEGER, BIGINT, HUGEINT
- UINTEGER, UBIGINT (unsigned)
- FLOAT, DOUBLE
- DECIMAL (precisão fixa)

**Texto:**
- VARCHAR, TEXT, STRING (equivalentes)

**Booleanos:**
- BOOLEAN, BOOL, LOGICAL

**Temporais:**
- DATE, TIME, TIMESTAMP
- TIMESTAMPTZ (com timezone)
- INTERVAL

**Estruturados:**
- BLOB (binário)
- UUID
- JSON
- ARRAY, LIST
- STRUCT (registros)
- MAP (dicionário)
- UNION (tipos mistos)

### 🔑 Pontos-Chave:
- DuckDB suporta **tipos complexos** nativamente
- **Listas** começam no índice 1 (não 0!)
- **STRUCT** permite dados hierárquicos
- **JSON** integrado no SQL
- **DECIMAL** para precisão monetária
- **TRY_CAST** para conversões seguras

### 💡 Boas Práticas:
1. Use DECIMAL para valores monetários
2. Use TIMESTAMP para logs
3. Use STRUCT para dados relacionados
4. Use LIST para coleções variáveis
5. Use TRY_CAST para evitar erros

### 📚 Próximo Capítulo:
Consultas em arquivos remotos!